### Prompt:
Imagine my data structure is:
- StateMachine has 
    - a dictionary of States
    - a variable pointer to State; 
    - a set of Void function pointers with different names
    - a set of variables of elementary type
- State has
    - a dictionary of Transitions
    - a set of Void function pointers with different names
    - a set of variables of elementary type}
- Transition has
    - a dictionary of void function pointers

1. First make the grammar for Lark in
2. Write a Python script and create the parser for this grammar.
3. Add an example to test

In [ ]:
from lark import Lark, Transformer, v_args

grammar = r"""
start: state_machine

state_machine: "StateMachine" "{" state_dictionary state_pointer void_func_ptrs elementary_vars "}"

state_dictionary: "States" "{" state+ "}"
state: "State" CNAME "{" transition_dictionary void_func_ptrs elementary_vars "}"
transition_dictionary: "Transitions" "{" transition+ "}"
transition: "Transition" CNAME "{" void_func_ptrs "}"

state_pointer: "StatePointer" CNAME ";"

void_func_ptrs: "VoidFunctions" "{" (CNAME ";")+ "}"
elementary_vars: "ElementaryVars" "{" (CNAME ";")+ "}"

%import common.CNAME
%import common.WS
%ignore WS
"""

# Apologies for the confusion, it appears that there's a mistake in the grammar. 
# The issue is caused by the transition rules, which expect at least one void function pointer 
# in a transition. To fix the problem, modify the transition rule in the "data_structure_grammar.lark" file to allow an empty set of void function pointers.

# Here's the updated grammar:


grammar = r"""
start: state_machine

state_machine: "StateMachine" "{" state_dictionary state_pointer void_func_ptrs elementary_vars "}"

state_dictionary: "States" "{" state+ "}"
state: "State" CNAME "{" transition_dictionary void_func_ptrs elementary_vars "}"
transition_dictionary: "Transitions" "{" transition+ "}"
transition: "Transition" CNAME "{" void_func_ptrs? "}"

state_pointer: "StatePointer" CNAME ";"

void_func_ptrs: "VoidFunctions" "{" (CNAME ";")* "}"
elementary_vars: "ElementaryVars" "{" (CNAME ";")* "}"

%import common.CNAME
%import common.WS
%ignore WS
"""

data_structure_parser = Lark(grammar, start="start", parser="lalr")

class DataStructureTransformer(Transformer):
    @v_args(inline=True)
    def state_machine(self, *items): #Had to change items -> *items
        return {"StateMachine": items}

    @v_args(inline=True)
    def state_dictionary(self, items):
        return {"States": items}

    @v_args(inline=True)
    def state(self, *items): # GPT helped to change items -> *items, 
                            # but later messed up with state_name = items[0].pop(0).value
                            # Some trace is below
        return {items[0]: items[1:]}
# #     def state(self, items):
# #         state_name = items.pop(0).value
# #         return {state_name: items}
# #     def state(self, *items):
# #         state_name = items[0].pop(0).value
# #         return {state_name: items[0]}
#     def state(self, *items): 
# #         items = items[0]  # Access the list within the tuple
# #         print('********',items)
#         state_name = items[0]#.pop(0).value
#         return {state_name: items[1:]}
    
    @v_args(inline=True)
    def transition_dictionary(self, items):
        return {"Transitions": items}

    @v_args(inline=True)
    def transition(self, items):
        return {items[0]: items[1:]}

    @v_args(inline=True)
    def state_pointer(self, item):
        return {"StatePointer": item}

    @v_args(inline=True)
    def void_func_ptrs(self, items):
        return {"VoidFunctions": items}

    @v_args(inline=True)
    def elementary_vars(self, items):
        return {"ElementaryVars": items}

def parse_data_structure(text):
    tree = data_structure_parser.parse(text)
    return DataStructureTransformer().transform(tree)

In [ ]:
example = """
StateMachine {
    States {
        State S1 {
            Transitions {
                Transition T1 {}
            }
            VoidFunctions {
                func1;
            }
            ElementaryVars {
                var1;
            }
        }
    }
    StatePointer S1;
    VoidFunctions {
        func2;
    }
    ElementaryVars {
        var2;
    }
}
"""

if __name__ == "__main__":
    result = parse_data_structure(example)
    print("Result is: \n", result)


### Prompt:
Write the Python script with
1. An example for a state machine, which has three states A, B and C and a void function called execute. 
2. Each of the states has transitions to the other two, a real variable called time and a function called action. 
3. Parse it with lark_parser and create the AST

In [ ]:
from lark import Lark, Transformer, v_args

# Updated grammar
grammar = """
start: state_machine

state_machine: "StateMachine" "{" state_dictionary state_pointer void_func_ptrs elementary_vars "}"

state_dictionary: "States" "{" state+ "}"
state: "State" CNAME "{" transition_dictionary void_func_ptrs elementary_vars "}"
transition_dictionary: "Transitions" "{" transition+ "}"
transition: "Transition" CNAME "{" void_func_ptrs? "}"

state_pointer: "StatePointer" CNAME ";"

void_func_ptrs: "VoidFunctions" "{" (CNAME ";")* "}"
elementary_vars: "ElementaryVars" "{" (real_var ";")* "}"

real_var: CNAME ":" "real"

%import common.CNAME
%import common.WS
%ignore WS
"""

data_structure_parser = Lark(grammar, start="start", parser="lalr")

class DataStructureTransformer(Transformer):
#     ChatGPT's suggestion:
#     def state_machine(self, items):
#         return {"StateMachine": items}
#     Correction:
    def state_machine(self, *items):
        return {"StateMachine": items}

    def state_dictionary(self, items):
        return {"States": items}

#     ChatGPT's suggestion:
#     def state(self, items):
#         items = items[0]
#         state_name = items.pop(0).value
#         return {state_name: items}
#     Correction:
    def state(self, *items):
        return {items[0]: items[1:]}

    def transition_dictionary(self, items):
        return {"Transitions": items}

    def transition(self, items):
        return {items[0]: items[1:]}

    def state_pointer(self, item):
        return {"StatePointer": item}

    def void_func_ptrs(self, items):
        return {"VoidFunctions": items}

    def elementary_vars(self, items):
        return {"ElementaryVars": items}

    def real_var(self, item):
        return {item[0]: "real"}

def parse_data_structure(text):
    tree = data_structure_parser.parse(text)
    return DataStructureTransformer().transform(tree)

# State machine example
example = """
StateMachine {
    States {
        State A {
            Transitions {
                Transition B {}
                Transition C {}
            }
            VoidFunctions {
                action;
            }
            ElementaryVars {
                time: real;
            }
        }
        State B {
            Transitions {
                Transition A {}
                Transition C {}
            }
            VoidFunctions {
                action;
            }
            ElementaryVars {
                time: real;
            }
        }
        State C {
            Transitions {
                Transition A {}
                Transition B {}
            }
            VoidFunctions {
                action;
            }
            ElementaryVars {
                time: real;
            }
        }
    }
    StatePointer A;
    VoidFunctions {
        execute;
    }
    ElementaryVars {}
}
"""

if __name__ == "__main__":
    result = parse_data_structure(example)
    print(result)


### Prompt:
Write a Python script and
1. Use llvmlite to generate IR code and the object file, where the data structures are instantiated
2. Use jinja2 to generate code and write the header file to allow using the variables in other c++ code.

# New attempt, with GPT40

Consider this grammar:

start: state_machine
state_machine: "StateMachine"  "{" state_array state_pointer void_func_ptrs elementary_vars "}"
state_array: "States" "{" state+ "}"
state: "State" CNAME "{" transition_array void_func_ptrs elementary_vars "}"
transition_array: "Transitions" "{" transition+ "}"
transition: "Transition" CNAME "{" void_func_ptrs? "}"
state_pointer: "StatePointer" CNAME ";"
void_func_ptrs: "VoidFunctions" "{" (CNAME ";")* "}"
elementary_vars: "ElementaryVars" "{" (real_var ";")* "}"
real_var: CNAME ":" "real"

%import common.CNAME
%import common.WS
%ignore WS

1. Check correctness as Lark grammar
2. Write a Python script and create the parser for this grammar.
3. Add an example to test

#### Output:
The provided grammar is correct as a Lark grammar.

To create a Python script and parser for the given grammar, follow the steps below:

Create a Python script named "state_machine_parser.py":

In [9]:
from lark import Lark, Transformer, v_args

grammar = """
start: state_machine
state_machine: "StateMachine"  "{" state_array state_pointer void_func_ptrs elementary_vars "}"
state_array: "States" "{" state+ "}"
state: "State" CNAME "{" transition_array void_func_ptrs elementary_vars "}"
transition_array: "Transitions" "{" transition+ "}"
transition: "Transition" CNAME "{" void_func_ptrs? "}"
state_pointer: "StatePointer" CNAME ";"
void_func_ptrs: "VoidFunctions" "{" (CNAME ";")* "}"
elementary_vars: "ElementaryVars" "{" (real_var ";")* "}"
real_var: CNAME ":" "real"

%import common.CNAME
%import common.WS
%ignore WS
"""

data_structure_parser = Lark(grammar, start="start", parser="lalr")

class DataStructureTransformer(Transformer):
    def state_machine(self, items):
        return {"StateMachine": items}

    def state_array(self, items):
        return {"States": items}

    def state(self, items): # Corrected items -> *items
#         items = items[0]
#         state_name = items.pop(0).value
#         return {state_name: items}
#         return {items[0]: items[1:]} # Corrected
        return {items.pop(0).value: items}

    def transition_array(self, items):
        return {"Transitions": items}

#     def transition(self, *items): # Corrected items -> *items
#         return {items[0]: items[1:]}
    def transition(self, items):
        return {items.pop(0).value: items}
    
    def state_pointer(self, item):
        return {"StatePointer": item}

    def void_func_ptrs(self, items):
        return {"VoidFunctions": items}

    def elementary_vars(self, items):
        return {"ElementaryVars": items}

    def real_var(self, item):
        return {item[0]: "real"}

def parse_state_machine(text):
    tree = data_structure_parser.parse(text)
    return DataStructureTransformer().transform(tree)


In [10]:
example = """
StateMachine {
    States {
        State A {
            Transitions {
                Transition B {}
                Transition C {}
            }
            VoidFunctions {
                action;
            }
            ElementaryVars {
                time: real;
            }
        }
        State B {
            Transitions {
                Transition A {}
                Transition C {}
            }
            VoidFunctions {
                action;
            }
            ElementaryVars {
                time: real;
            }
        }
        State C {
            Transitions {
                Transition A {}
                Transition B {}
            }
            VoidFunctions {
                action;
            }
            ElementaryVars {
                time: real;
            }
        }
    }
    StatePointer A;
    VoidFunctions {
        execute;
    }
    ElementaryVars {}
}
"""

if __name__ == "__main__":
    result = parse_state_machine(example)
    print("Result is: \n", result)


Result is: 
 Tree('start', [{'StateMachine': [{'States': [{'A': [{'Transitions': [{'B': []}, {'C': []}]}, {'VoidFunctions': [Token('CNAME', 'action')]}, {'ElementaryVars': [{Token('CNAME', 'time'): 'real'}]}]}, {'B': [{'Transitions': [{'A': []}, {'C': []}]}, {'VoidFunctions': [Token('CNAME', 'action')]}, {'ElementaryVars': [{Token('CNAME', 'time'): 'real'}]}]}, {'C': [{'Transitions': [{'A': []}, {'B': []}]}, {'VoidFunctions': [Token('CNAME', 'action')]}, {'ElementaryVars': [{Token('CNAME', 'time'): 'real'}]}]}]}, {'StatePointer': [Token('CNAME', 'A')]}, {'VoidFunctions': [Token('CNAME', 'execute')]}, {'ElementaryVars': []}]}])


### Prompt:
Write the Python script with
1. An example for a state machine, which has three states A, B and C and a void function called execute. 
2. Each of the states has transitions to the other two, a real variable called time and a function called action. 
3. Parse it with lark_parser and create the AST

In [11]:
from lark import Lark, Transformer, v_args

grammar = """
start: state_machine
state_machine: "StateMachine"  "{" state_array state_pointer void_func_ptrs elementary_vars "}"
state_array: "States" "{" state+ "}"
state: "State" CNAME "{" transition_array void_func_ptrs elementary_vars "}"
transition_array: "Transitions" "{" transition+ "}"
transition: "Transition" CNAME "{" void_func_ptrs? "}"
state_pointer: "StatePointer" CNAME ";"
void_func_ptrs: "VoidFunctions" "{" (CNAME ";")* "}"
elementary_vars: "ElementaryVars" "{" (real_var ";")* "}"
real_var: CNAME ":" "real"

%import common.CNAME
%import common.WS
%ignore WS
"""

data_structure_parser = Lark(grammar, start="start", parser="lalr")

class DataStructureTransformer(Transformer):
    def state_machine(self, items):
        return {"StateMachine": items}

    def state_array(self, items):
        return {"States": items}

    def state(self, items):
        state_name = items.pop(0).value
        return {state_name: items}

    def transition_array(self, items):
        return {"Transitions": items}

    def transition(self, items):
        return {items.pop(0).value: items}

    def state_pointer(self, item):
        return {"StatePointer": item}

    def void_func_ptrs(self, items):
        return {"VoidFunctions": items}

    def elementary_vars(self, items):
        return {"ElementaryVars": items}

    def real_var(self, item):
        return {item[0]: "real"}

def parse_state_machine(text):
    tree = data_structure_parser.parse(text)
    return DataStructureTransformer().transform(tree)

# State machine example
example = """
StateMachine {
    States {
        State A {
            Transitions {
                Transition B {}
                Transition C {}
            }
            VoidFunctions {
                action;
            }
            ElementaryVars {
                time: real;
            }
        }
        State B {
            Transitions {
                Transition A {}
                Transition C {}
            }
            VoidFunctions {
                action;
            }
            ElementaryVars {
                time: real;
            }
        }
        State C {
            Transitions {
                Transition A {}
                Transition B {}
            }
            VoidFunctions {
                action;
            }
            ElementaryVars {
                time: real;
            }
        }
    }
    StatePointer A;
    VoidFunctions {
        execute;
    }
    ElementaryVars {}
}
"""

if __name__ == "__main__":
    result = parse_state_machine(example)
    print("Result is: \n", result)


Result is: 
 Tree('start', [{'StateMachine': [{'States': [{'A': [{'Transitions': [{'B': []}, {'C': []}]}, {'VoidFunctions': [Token('CNAME', 'action')]}, {'ElementaryVars': [{Token('CNAME', 'time'): 'real'}]}]}, {'B': [{'Transitions': [{'A': []}, {'C': []}]}, {'VoidFunctions': [Token('CNAME', 'action')]}, {'ElementaryVars': [{Token('CNAME', 'time'): 'real'}]}]}, {'C': [{'Transitions': [{'A': []}, {'B': []}]}, {'VoidFunctions': [Token('CNAME', 'action')]}, {'ElementaryVars': [{Token('CNAME', 'time'): 'real'}]}]}]}, {'StatePointer': [Token('CNAME', 'A')]}, {'VoidFunctions': [Token('CNAME', 'execute')]}, {'ElementaryVars': []}]}])


### Prompt:
Write a Python script and
1. Use llvmlite to generate IR code and the object file, where the data structures are instantiated
2. Use jinja2 to generate code and write the header file to allow using the variables in other c++ code.

In [18]:
import llvmlite.ir as ir
import llvmlite.binding as llvm
from jinja2 import Template

# State machine structure from the previous example
state_machine = {
    'StateMachine': [
        {'States': [
            {'A': [
                {'Transitions': ['B', 'C']},
                {'VoidFunctions': ['action']},
                {'ElementaryVars': [{'time': 'real'}]}
            ]},
            {'B': [
                {'Transitions': ['A', 'C']},
                {'VoidFunctions': ['action']},
                {'ElementaryVars': [{'time': 'real'}]}
            ]},
            {'C': [
                {'Transitions': ['A', 'B']},
                {'VoidFunctions': ['action']},
                {'ElementaryVars': [{'time': 'real'}]}
            ]}
        ]},
        {'StatePointer': 'A'},
        {'VoidFunctions': ['execute']},
        {'ElementaryVars': []}
    ]
}

# Generate LLVM IR code
module = ir.Module(name="state_machine")
double_type = ir.DoubleType()

# Generate global variables for states and state pointer
state_pointer = ir.GlobalVariable(module, double_type, "state_pointer")
state_pointer.linkage = 'external'
state_pointer.initializer = None

states = {}
for state in state_machine["StateMachine"][0]["States"]:
    state_name = list(state.keys())[0]
    variable = ir.GlobalVariable(module, double_type, f"{state_name}_time")
    variable.linkage = 'global'
    variable.initializer = ir.Constant(double_type, 0)
    states[state_name] = variable

# Generate object file
llvm.initialize()
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()

target_machine = llvm.Target.from_default_triple().create_target_machine()
llvm_ir = str(module)
llvm_module = llvm.parse_assembly(llvm_ir)

with open("state_machine.o", "wb") as f:
    f.write(target_machine.emit_object(llvm_module))

# Generate C++ header file
header_template = """
#ifndef STATE_MACHINE_H
#define STATE_MACHINE_H

extern double state_pointer;
{% for state in states %}
extern double {{ state }}_time;
{% endfor %}

#endif // STATE_MACHINE_H
"""

template = Template(header_template)
header = template.render(states=states.keys())

with open("state_machine.h", "w") as f:
    f.write(header)


RuntimeError: LLVM IR parsing error
<string>:6:20: error: expected type
@"A_time" = global global double              0x0
                   ^
